# Item Response Ranking for IRT

This notebook will show you how to train and use the IRR-IRT.
Refer to [IRR doc](../../docs/IRR.md) for more details.
First, we will show how to get the data (here we use a0910 as the dataset).
Then we will show how to train a IRR-IRT and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

In [1]:
import logging
from longling.lib.structure import AttrDict
from longling import set_logging_info
from EduCDM.IRR import pair_etl as etl, point_etl as vt_etl, extract_item

set_logging_info()

params = AttrDict(
    batch_size=256,
    n_neg=10,
    n_imp=10,
    logger=logging.getLogger(),
    hyper_params={"user_num": 4164}
)
item_knowledge = extract_item("../../data/a0910/item.csv", 123, params)
train_data, train_df = etl("../../data/a0910/train.csv", item_knowledge, params)
valid_data, _ = vt_etl("../../data/a0910/valid.csv", item_knowledge, params)
test_data, _ = vt_etl("../../data/a0910/test.csv", item_knowledge, params)

train_data, valid_data, test_data

rating2triplet: 100%|██████████| 17051/17051 [00:16<00:00, 1032.23it/s]


(<longling.lib.iterator.AsyncLoopIter at 0x1f62c7c49a0>,
 <torch.utils.data.dataloader.DataLoader at 0x1f62bae1be0>)

In [2]:
train_df

,user_id,item_id,score
0,1615,12977,1.0
1,782,13124,0.0
2,1084,16475,0.0
3,593,8690,0.0
4,127,14225,1.0
...,...,...,...
186044,2280,6019,0.0
186045,121,2,1.0
186046,601,5425,1.0
186047,573,2412,0.0


In [3]:
from EduCDM.IRR import IRT

cdm = IRT(
    4163 + 1,
    17746 + 1,
    123
)
cdm.train(
    train_data,
    valid_data,
    epoch=2,
)
cdm.save("IRR-IRT.params")

Epoch 0: 727it [00:48, 14.99it/s]
formatting item df: 100%|██████████| 10415/10415 [00:00<00:00, 11426.27it/s]
ranking metrics: 10415it [00:15, 674.53it/s]
formatting item df: 100%|██████████| 10415/10415 [00:00<00:00, 12475.22it/s]
ranking metrics: 10415it [00:14, 721.48it/s]
INFO:root:save parameters to IRR-IRT.params


[Epoch 0] Loss: 2.573528, PointLoss: 0.675686, PairLoss: 4.471370
[Epoch 0]
      ndcg@k  precision@k  recall@k      f1@k     len@k  support@k
1   1.000000     0.666635  0.469846  0.520679  1.000000      10415
3   0.886982     0.671884  0.734436  0.683024  1.906961      10415
5   0.890280     0.673186  0.792938  0.710393  2.229573      10415
10  0.890548     0.673845  0.815712  0.719816  2.423428      10415
auc: 0.830947	map: 0.908118	mrr: 0.898518	coverage_error: 3.039120	ranking_loss: 0.295783	len: 2.458569	support: 10415
[Epoch 1] Loss: 2.559691, PointLoss: 0.655644, PairLoss: 4.463738
[Epoch 1]
      ndcg@k  precision@k  recall@k      f1@k     len@k  support@k
1   1.000000     0.668459  0.470703  0.521785  1.000000      10415
3   0.887368     0.672428  0.734589  0.683341  1.906961      10415
5   0.890856     0.673321  0.792951  0.710456  2.229573      10415
10  0.891069     0.673961  0.815888  0.719951  2.423428      10415
auc: 0.832663	map: 0.908796	mrr: 0.899676	coverage_error: 3

In [4]:
cdm.load("IRR-IRT.params")
print(cdm.eval(test_data))

INFO:root:load parameters from IRR-IRT.params
formatting item df: 100%|██████████| 13682/13682 [00:01<00:00, 13153.81it/s]
ranking metrics: 13682it [00:23, 584.22it/s]


      ndcg@k  precision@k  recall@k      f1@k     len@k  support@k
1   1.000000     0.671466  0.370265  0.434827  1.000000      13682
3   0.862770     0.668872  0.663951  0.634433  2.268528      13682
5   0.868876     0.667471  0.769999  0.689557  2.981582      13682
10  0.869360     0.667608  0.844664  0.723544  3.723652      13682
auc: 0.767649	map: 0.869138	mrr: 0.873835	coverage_error: 4.662276	ranking_loss: 0.320187	len: 4.075428	support: 13682
